In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
#Welcome to my notebook about Titanic contest. 
#First of all, let's save the datasets into variables in order to use it later.
train_titanic = pd.read_csv('/kaggle/input/titanic/train.csv')
test_titanic = pd.read_csv('/kaggle/input/titanic/test.csv')
submission_template = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [3]:
#The first step within any project of Machine Learning, is read, analyse and understand the datasets.
#I can see that the columns called 'Age' and 'Cabin', have missing values (nan or empty), so we need to clean data or take decisions about the features for our model
train_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
#Let's split our data.
#The prediction target is: 'Survived'
y = train_titanic.Survived

#We need to choose the feature data in order to train the model.
feature_data = ['Pclass','Sex','Age','SibSp','Parch']
X = train_titanic[feature_data]

In [5]:
#I'm going to search missing data, before i looked into all the data set with .info(), i realized that 'Survived' data are complete.
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    object 
 2   Age     714 non-null    float64
 3   SibSp   891 non-null    int64  
 4   Parch   891 non-null    int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 34.9+ KB


In [6]:
#From the analysis before, we found that row 'Age' have some missing data, let's get rid of it
print(X[X.isnull().values.any(axis=1)])

     Pclass     Sex  Age  SibSp  Parch
5         3    male  NaN      0      0
17        2    male  NaN      0      0
19        3  female  NaN      0      0
26        3    male  NaN      0      0
28        3  female  NaN      0      0
..      ...     ...  ...    ...    ...
859       3    male  NaN      0      0
863       3  female  NaN      8      2
868       3    male  NaN      0      0
878       3    male  NaN      0      0
888       3  female  NaN      1      2

[177 rows x 5 columns]


In [7]:
#The total numbers of rows is: 891 and the number of rows with missing data is: 177, if we look at the .info() of the whole data, that number is consistent, that means we already know that the missing values are NaN.
#We have two options, just eliminate all the rows with missing data or replace the NaN with some consistent data.
